## Activation Generator

In [ ]:
from lib.CLIC_dataset import build_trainloader
import torch
from tqdm import tqdm
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"
import torch.optim as optim
import torch
from lib.student_v1 import StudentEncoderPruned_V1
from loss import get_loss_functions
import torch.optim as optim
from CLIC_dataset import build_trainloader

import NeuralCompression.neuralcompression.functional as ncF
device = "cuda" if torch.cuda.is_available() else "cpu"

../Objects/typeobject.c:4243: type_traverse: Assertion failed: type_traverse() called on non-heap type 'MultiLabelSoftMarginLoss'
Enable tracemalloc to get the memory block allocation traceback

object address  : 0x5893a86eaa10
object refcount : 2
object type     : 0x58938d6619a0
object type name: type
object repr     : <class 'MultiLabelSoftMarginLoss'>

Fatal Python error: _PyObject_AssertFailed: _PyObject_AssertFailed
Python runtime state: initialized

Thread 0x00007bc8757fa640 (most recent call first):
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/parentpoller.py", line 40 in run
  File "/usr/lib/python3.10/threading.py", line 1016 in _bootstrap_inner
  File "/usr/lib/python3.10/threading.py", line 973 in _bootstrap

Thread 0x00007bc875ffb640 (most recent call first):
  File "/usr/lib/python3.10/threading.py", line 320 in wait
  File "/usr/lib/python3.10/threading.py", line 607 in wait
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/history.py", line 894 in

In [ ]:
model = torch.hub.load("facebookresearch/NeuralCompression", "msillm_quality_3", force_reload=True)
model = model.eval()
model.update()
model.update_tensor_devices("compress")

# Freeze Model
for p in model.parameters():
    p.requires_grad = False


In [3]:
# Feature storage for hints
teacher_feats = {}

# Assuming teacher.encoder layers accessible as .conv1, .conv2, etc.
def get_teacher_hook(name):
    def hook(module, input, output):
        teacher_feats[name] = output.detach().cpu()
    return hook
# Register hooks on desired hint layers
model.encoder.blocks[0].register_forward_hook(get_teacher_hook('hint1'))
model.encoder.blocks[1].register_forward_hook(get_teacher_hook('hint2'))
model.encoder.blocks[2].register_forward_hook(get_teacher_hook('hint3'))
model.encoder.blocks[3].register_forward_hook(get_teacher_hook('hint4'))
model.encoder.blocks[4].register_forward_hook(get_teacher_hook('hint5'))

In [5]:
train_loader = build_trainloader(batch_size=128, img_dir="/workspace/unmounted/dataset/CLIC/val2017")

In [ ]:
model.encoder.to(device)

for batch_idx, images in enumerate(tqdm(train_loader)):
    with torch.no_grad():
        
        images, (_, _) = ncF.pad_image_to_factor(images, model._factor)
        images = images.to(device)
        
        current_batch_activations = {}
        current_batch_activations['images'] = images.detach().cpu()

        y_final = model.encoder(images)
        current_batch_activations['layer6'] = y_final.detach().cpu()
        
        # Store teacher features
        current_batch_activations['layer1'] = teacher_feats['hint1'].detach().cpu()
        current_batch_activations['layer2'] = teacher_feats['hint2'].detach().cpu()
        current_batch_activations['layer3'] = teacher_feats['hint3'].detach().cpu()
        current_batch_activations['layer4'] = teacher_feats['hint4'].detach().cpu()
        current_batch_activations['layer5'] = teacher_feats['hint5'].detach().cpu()

        # Save current batch activations to separate file
        #np.savez(f'/workspace/main/CLIC_activations/ILLM_Q3/batch_{batch_idx:04d}.npz', **current_batch_activations)
        torch.save(current_batch_activations, f'/workspace/unmounted/CLIC_activations/ILLM_Q3_B128_torch/batch_{batch_idx:04d}.pt')

 38%|███▊      | 15/40 [01:18<02:27,  5.90s/it]

In [8]:
from CLIC_dataset import build_activation_dataloader

loader = build_activation_dataloader(npz_dir="/workspace/unmounted/CLIC_activations/ILLM_Q3")

In [9]:
from student_v1 import StudentEncoderBase_V2

In [10]:
for batch in loader:
    print(batch['images'].shape)
    print(batch['layer1'].shape)
    print(batch['layer2'].shape)
    print(batch['layer3'].shape)
    print(batch['layer4'].shape)
    print(batch['layer5'].shape)
    print(batch['layer6'].shape)
    break
    

torch.Size([32, 3, 224, 224])
torch.Size([32, 60, 224, 224])
torch.Size([32, 120, 112, 112])
torch.Size([32, 240, 56, 56])
torch.Size([32, 480, 28, 28])
torch.Size([32, 960, 14, 14])
torch.Size([32, 220, 14, 14])


In [11]:
from tqdm import tqdm
import matplotlib.pyplot as plt

model.decoder.to(device)
student = StudentEncoderBase_V2()
student.to(device)
optimizer = optim.Adam(student.parameters(), lr=0.0085)
msssim_loss, vgg_perceptual, distillation_loss = get_loss_functions()
vgg_perceptual = vgg_perceptual.to(device)



def train_epoch(dataloader, epoch=None):
    student.train()
    running_loss = 0.0
    total_hint1_loss = 0.0
    total_hint2_loss = 0.0
    total_latent_loss = 0.0
    total_ssim_loss = 0.0
    total_perc_loss = 0.0


    student.to(device)
    # Add TQDM loader
    loop = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch if epoch is not None else ''}")

    for i, batch in loop:
        
        x = batch["images"]
        x = x.to(device)
        # Padding Correction        
        x, (_, _) = ncF.pad_image_to_factor(x, model._factor)
         
        optimizer.zero_grad()

        print("Input Shape: ", x.shape)
        
        y = student.block1(x)
        print(y.shape)

        z = model.encoder.blocks(0)(x)

        print(z.shape)
        
        print(batch['layer1'].shape)
        hint1_loss = distillation_loss(y.to(device), batch['layer1'].to(device))

        y = student.block2(y)
        hint2_loss = distillation_loss(y.to(device), batch['layer2'].to(device))
        
        y = student.block3(y)
        hint3_loss = distillation_loss(y.to(device), batch['layer3'].to(device))
        
        y = student.block4(y)
        hint4_loss = distillation_loss(y.to(device), batch['layer4'].to(device))
        
        y = student.block5(y)
        hint5_loss = distillation_loss(y, batch['layer5'])
        
        y = student.block6(y)
        latent_loss = distillation_loss(y, batch['layer6'])

        
        #Reconstruction via Teacher Decoder
        with torch.no_grad():            
            x_recon = model.decoder(y)
            #model.decoder.to("cpu")
            
        perc_loss = vgg_perceptual(x, x_recon)
        #ssim_loss = msssim_loss(x, x_recon)

        # Cumulative Loss
        loss = (1.0 * hint1_loss
                + 1.0 * hint2_loss
                + 1.0 * hint3_loss
                + 1.0 * hint4_loss
                + 1.0 * hint5_loss
                + 1.0 * latent_loss
                #+ gamma_msssim * ssim_loss
                + gamma_perc * perc_loss)

        # Backprop and optimize
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        
        # Accumulate loss values
        running_loss += loss.item() * x.size(0)
        total_hint1_loss += hint1_loss.item() * x.size(0)
        total_hint2_loss += hint2_loss.item() * x.size(0)
        total_latent_loss += latent_loss.item() * x.size(0)
        #total_ssim_loss += ssim_loss.item() * x.size(0)
        total_perc_loss += perc_loss.item() * x.size(0)

    # Average losses
    dataset_size = len(dataloader.dataset)
    epoch_loss = running_loss / dataset_size
    avg_hint1_loss = total_hint1_loss / dataset_size
    avg_hint2_loss = total_hint2_loss / dataset_size
    avg_latent_loss = total_latent_loss / dataset_size
    #avg_ssim_loss = total_ssim_loss / dataset_size
    avg_perc_loss = total_perc_loss / dataset_size

    print(f"\n[Epoch {epoch}] Component-wise Losses:")
    print(f"Hint1 Loss:     {avg_hint1_loss:.4f}")
    print(f"Hint2 Loss:     {avg_hint2_loss:.4f}")
    print(f"Latent Loss:    {avg_latent_loss:.4f}")
    #print(f"MS-SSIM Loss:   {avg_ssim_loss:.4f}")
    print(f"Perceptual Loss:{avg_perc_loss:.4f}")
    print(f"Total Loss:     {epoch_loss:.4f}")

    # losses_per_epoch[] umer shall continue

    # Plot reconstructed image after the epoch
    x_vis = x[0].detach().cpu().permute(1, 2, 0)
    x_recon_vis = x_recon[0].detach().cpu().permute(1, 2, 0)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(x_vis)
    axs[0].set_title("Original Image")
    axs[0].axis('off')

    axs[1].imshow(x_recon_vis)
    axs[1].set_title("Reconstructed Image")
    axs[1].axis('off')

    plt.suptitle(f"Reconstruction at Epoch {epoch}")
    plt.show()

    return epoch_loss


In [12]:
    train_loss = train_epoch(loader)

Epoch :   0%|          | 0/157 [00:00<?, ?it/s]

Input Shape:  torch.Size([32, 3, 256, 256])
torch.Size([32, 60, 256, 256])


TypeError: conv2d() received an invalid combination of arguments - got (int, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!int!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
